<a href="https://colab.research.google.com/github/malipalema/lafand-mt/blob/main/masakhane_baseline_on_lafand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating the Masakhane MT models on the English to Isizulu LAFAND test set

## Masakhane - Machine Translation for African Languages (Using JoeyNMT)

In [ ]:
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3224, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 3224 (delta 155), reused 209 (delta 130), pack-reused 2951
Receiving objects: 100% (3224/3224), 8.19 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (2183/2183), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
     |████████████████████████████████| 413 kB 42.8 MB/s 
     |████████████████████████████████| 250 kB 41.6 MB/s

In [1]:
import torch
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device_num = torch.cuda.current_device()
torch.cuda.get_device_name(device_num)
# torch.cuda.is_available()

'Tesla K80'

In [2]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 80 kB 3.8 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import numpy as np
import pandas as pd

source_language = "en"
target_language = "yo" 
lc = True  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted
vocab_size=4000
corpus = "JW300"

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag
os.environ["vocab_size"] = str(vocab_size)
os.environ["corpus"] = corpus

In [5]:
# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/baseline/$src-$trg-$tag"
gdrive_path = f"/content/drive/My Drive/masakhane/baseline/{source_language}-{target_language}-{tag}/"
os.environ["gdrive_path"] = gdrive_path
! echo $gdrive_path

/content/drive/My Drive/masakhane/baseline/en-yo-baseline/


In [6]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/baseline/en-yo-baseline/


In [7]:
# Download the global test set.
! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-12-03 16:07:14--  https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.03s   

2021-12-03 16:07:14 (8.70 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2021-12-03 16:07:14--  https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-yo.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200

In [8]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
blanks = [] # sometimes blank lines creep innto test set - store which lines these are
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    if len(line)<=1:
      blanks.append(j)
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))
print(f'There are {len(blanks)} blank lines in the test set')

Loaded 3571 global test sentences to filter from the training/dev data.
There are 0 blank lines in the test set


In [9]:
# filter test set

source_file = f"test.{source_language}"
target_file = f"test.{target_language}"

source = []
target = []

with open(source_file) as f:
  source = f.readlines()
            
with open(target_file) as f:
  target = f.readlines()

df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

# remove trailing newline chars
df['source_sentence'] = df['source_sentence'].str.rstrip('" \n')
df['target_sentence'] = df['target_sentence'].str.rstrip('" \n')

# remove leading newline chars
df['source_sentence'] = df['source_sentence'].str.lstrip('"')
df['target_sentence'] = df['target_sentence'].str.lstrip('"')

# remove rows with really short sentences
df = df[~(df['source_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters
df = df[~(df['target_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters

# save the filtered test set
df['source_sentence'].to_csv(f'{source_file}', index=False, header=False, doublequote=False)
df['target_sentence'].to_csv(f'{target_file}', index=False, header=False, doublequote=False)

In [10]:
df.head()

,source_sentence,target_sentence
0,Some names in this article have been changed .,A ti yí àwọn orúkọ kan padà nínú àpilẹ̀kọ yìí .
1,It does not belong to man who is walking even ...,Kì í ṣe ti ènìyàn tí ń rìn àní láti darí àwọn ...
2,Published by Jehovah’s Witnesses but now out o...,"Àwọn Ẹlẹ́rìí Jèhófà ló tẹ̀ ẹ́ jáde , àmọ́ wọn ..."
3,“ The whole world is lying in the power of the...,“ Gbogbo ayé wà lábẹ́ agbára ẹni burúkú náà . ”
4,"Moreover , do not call anyone your father on e...","Jù bẹ́ẹ̀ lọ , ẹ má pe ẹnikẹ́ni ní baba yín lór..."


In [16]:
# en_jw = read_file(data_dir + 'jw300.en')
# zu_jw = read_file(data_dir + 'jw300.zu')

menyo_train = pd.read_csv('/content/drive/MyDrive/masakhane/train.tsv', sep='\t')
en_menyo_train = menyo_train['English'].values
yo_menyo_train = menyo_train['Yoruba'].values

menyo_dev = pd.read_csv('/content/drive/MyDrive/masakhane/dev.tsv', sep='\t')
en_menyo_dev = menyo_dev['english'].values
yo_menyo_dev = menyo_dev['yoruba'].values

menyo_test = pd.read_csv('/content/drive/MyDrive/masakhane/test_news.tsv', sep='\t')
en_menyo_test = menyo_test['english'].values
yo_menyo_test = menyo_test['yoruba'].values

# merge data
# Train data
train_data_en = list(en_menyo_train)  # en_jw + list(en_menzu_train)
train_data_yo = list(yo_menyo_train)  # zu_jw + list(zu_menzu_train)

df_train_enyo = pd.DataFrame(train_data_en, columns=['source_sentence'])
df_train_enyo['target_sentence'] = train_data_yo

df_train_yoen = pd.DataFrame(train_data_yo, columns=['source_sentence'])
df_train_yoen['target_sentence'] = train_data_en

# dev data
df_dev_enyo = pd.DataFrame(en_menyo_dev, columns=['source_sentence'])
df_dev_enyo['target_sentence'] = yo_menyo_dev

df_dev_yoen = pd.DataFrame(yo_menyo_dev, columns=['source_sentence'])
df_dev_yoen['target_sentence'] = en_menyo_dev

# test data
df_test_enyo = pd.DataFrame(en_menyo_test, columns=['source_sentence'])
df_test_enyo['target_sentence'] = yo_menyo_test

df_test_yoen = pd.DataFrame(yo_menyo_test, columns=['source_sentence'])
df_test_yoen['target_sentence'] = en_menyo_test

In [18]:
df_test_enyo.shape

(3102, 2)

In [19]:
# How many samples
size = len(df_test_enyo)
print(f"\n {size} samples in original text")


 3102 samples in original text


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [21]:
## Preprocessing - Step 1 : Drop NaNs

df_pp = df_test_enyo.dropna()
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping all NaNs")
size = new_size


 0(0.00 %) samples removed by dropping all NaNs


In [22]:
## Preprocessing - Step 2a : Drop all duplicates in Source (en) text

df_pp = df_pp.drop_duplicates(subset='source_sentence')
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Source sentence duplicates")
size = new_size


 45(1.45 %) samples removed by dropping Source sentence duplicates


In [23]:
## Preprocessing - Step 2b : Drop all duplicates in Target (zu) text

df_pp = df_pp.drop_duplicates(subset='target_sentence')
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Target sentence duplicates")
size = new_size


 0(0.00 %) samples removed by dropping Target sentence duplicates


In [24]:
##  Preprocessing - Step 3 : Remove all numeric entries

pattern = r"([0-9]*\.?[0-9]*)"  # catch integers and decimals
import re
r = re.compile(pattern)

In [25]:
%%time
##  Preprocessing - Step 3a : Remove all numeric entries - Source text

df_pp['source_sentence'] = df_pp['source_sentence'].str.replace(pattern,"")
df_pp['source_sentence'] = df_pp['source_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from source text")
size = new_size


 0(0.00 %) samples removed by dropping nummeric entries from source text
CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 132 ms


In [26]:
# DOnt RUN
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 50 kB 2.9 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149870 sha256=15cacaa74c656077591967f862d26ec7be0ef5edf033cedb9f1c56cf59088d45
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [27]:
# Dont RUN
start_time = time.time()
### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

with Pool(cpu_count()-1) as pool:
    scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
hours, rem = divmod(time.time() - start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# Filter out "almost overlapping samples"
df_pp = df_pp.assign(scores=scores)
df_pp = df_pp[df_pp['scores'] < 95]

done in 0.0h:0.0min:55.06313967704773seconds


In [28]:
# Dont Run
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# TODO: if your corpus is smaller than 1000, reduce this number. With a corpus that small you might not obtain good results with NMT though :/
# Do the split between dev/train and create parallel corpora
num_dev_patterns = 100

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=aFalse)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# TODO: Doublecheck the format below. There should be no extra quotation marks or weird characters. It should also not be empty.
! head train.*
! head dev.*

106

151

37

45

180

...

146

211

138

169

==> train.en <==
[update] as members of sebin searched diaz's and soto's home, luz mely reyes reported live from the scene
the sntp also reported the incident:
urgent at this time, : am, a commission from the intelligence forces arrives to journalist and human rights activist luis carlos díaz's home, missing since : pm #whereisluiscarlos
other human rights and freedom of speech organizations joined the campaign through #dondeestaluiscarlos (where is luis carlos?) which is, at the moment, a trending topic in venezuela's twittosphere
díaz is a journalist and a human rights and freedom of speech advocate who is well known and highly appreciated in venezuela and abroad for his commentary and criticism of the government of nicolas maduro
he has long worked with soto producing web-based video and radio programs focused on politics and human rights in venezuela
he has also worked as an educator and promoter of the creation of citizen media spaces and independent media projects
díaz has also b

In [29]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
#! pip install torch==1.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'joeynmt'...
remote: Enumerating objects: 3224, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 3224 (delta 155), reused 209 (delta 130), pack-reused 2951
Receiving objects: 100% (3224/3224), 8.19 MiB | 16.34 MiB/s, done.
Resolving deltas: 100% (2183/2183), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 33.6 MB/s 
     |████████████████████████████████| 413 kB 39.1 MB/s 
     |████████████████████████████████| 103 kB 47.5 MB/s

In [30]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p "$data_path"
! cp train.* "$data_path"
! cp test.* "$data_path"
! cp dev.* "$data_path"
! cp bpe.codes.4000 "$data_path"
! ls "$data_path"

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Test language Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.yo     test.yo	   train.en
dev.bpe.en	dev.yo	     test.en	     train.bpe.en  train.yo
dev.bpe.yo	test.bpe.en  test.en-any.en  train.bpe.yo
cp: target '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/' is not a directory
cp: target '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/' is not a directory
cp: target '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/' is not a directory
cp: cannot create regular file '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/': Not a directory
ls: cannot access '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/': No such file or directory
BPE Test language Sentences
A@@ p@@ ata ńlá ti ìgb@@ àgb@@ ó@@ ̣ ( W@@ o ìpín@@ r@@ ò@@ ̣ 1@@ 2 - 1@@ 4 )
À@@ ṣí@@ borí ìgb@@ àl@@ à ( W@@ o ìpín@@ r@@ ò@@ ̣ 1@@ 5 - 1@@ 8 )
"@@ M@@ o ti rí i pé àwọn èè@@ yàn máa ń f@@ é@@ ̣ gbó@@ ̣@@ r@@ ò@@ ̣ wa tí w@@ ó@@ ̣@@ n bá rí i pé a ló@@ ye B@@ í@@ b@@ é@@ l@@ ì dá@@ adá@@ a , a sì f@@ é@@ ̣

In [31]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

cp: target '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/' is not a directory
cp: target '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/' is not a directory
cp: target '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/' is not a directory
cp: cannot create regular file '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/': Not a directory
ls: cannot access '/content/drive/My Drive/masakhane/baseline/en-yo-baseline/': No such file or directory


In [32]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "none"            # `tokenize` option in sacrebleu.corpus_bleu() function (options include: "none" (use for already tokenized test data), "13a" (default minimal tokenizer), "intl" which mostly does punctuation and unicode, etc) 

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 25
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 100          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 1

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

3514

In [33]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2021-12-03 16:27:43,566 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-12-03 16:27:43,581 - INFO - joeynmt.data - Loading training data...
2021-12-03 16:27:43,636 - INFO - joeynmt.data - Building vocabulary...
2021-12-03 16:27:43,940 - INFO - joeynmt.data - Loading dev data...
2021-12-03 16:27:43,942 - INFO - joeynmt.data - Loading test data...
2021-12-03 16:27:44,029 - INFO - joeynmt.data - Data loaded.
2021-12-03 16:27:44,030 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-03 16:27:44,461 - INFO - joeynmt.model - Enc-dec model built.
2021-12-03 16:27:46,849 - INFO - joeynmt.training - Total params: 12107520
2021-12-03 16:27:46,851 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-12-03 16:27:58,516 - INFO - joeynmt.helpers - cfg.name                           : enyo_transformer
2021-12-03 16:27:58,516 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-12-03 16:27:58,

In [34]:
# Copy the created models from the notebook storage to google drive for persistent storage 
! mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"
! cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [35]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 100	Loss: 25717.04688	PPL: 403.52728	bleu: 0.03219	LR: 0.00030000	*
Steps: 200	Loss: 25040.62695	PPL: 344.61349	bleu: 0.01046	LR: 0.00030000	*
Steps: 300	Loss: 24206.30859	PPL: 283.65601	bleu: 0.01047	LR: 0.00030000	*
Steps: 400	Loss: 23636.44727	PPL: 248.34116	bleu: 0.02877	LR: 0.00030000	*
Steps: 500	Loss: 23217.34375	PPL: 225.20686	bleu: 0.09617	LR: 0.00030000	*
Steps: 600	Loss: 22636.63281	PPL: 196.67035	bleu: 0.12135	LR: 0.00030000	*
Steps: 700	Loss: 22100.86328	PPL: 173.56023	bleu: 0.17651	LR: 0.00030000	*
Steps: 800	Loss: 21621.75391	PPL: 155.20390	bleu: 0.35282	LR: 0.00030000	*
Steps: 900	Loss: 21157.94922	PPL: 139.28551	bleu: 0.44453	LR: 0.00030000	*
Steps: 1000	Loss: 20708.88867	PPL: 125.43050	bleu: 0.39353	LR: 0.00030000	*
Steps: 1100	Loss: 20508.19531	PPL: 119.69261	bleu: 0.22327	LR: 0.00030000	*
Steps: 1200	Loss: 20269.20117	PPL: 113.20101	bleu: 0.39414	LR: 0.00030000	*
Steps: 1300	Loss: 20065.06641	PPL: 107.93581	bleu: 0.52607	LR: 0.00030000	*
Steps: 1400	Loss: 199

In [36]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2021-12-03 16:52:53,708 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-12-03 16:52:53,709 - INFO - joeynmt.data - Building vocabulary...
2021-12-03 16:52:53,990 - INFO - joeynmt.data - Loading dev data...
2021-12-03 16:52:53,993 - INFO - joeynmt.data - Loading test data...
2021-12-03 16:52:54,027 - INFO - joeynmt.data - Data loaded.
2021-12-03 16:52:54,034 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-12-03 16:52:54,034 - INFO - joeynmt.prediction - Loading model from models/enyo_transformer/1900.ckpt
2021-12-03 16:52:56,744 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-12-03 16:52:56,991 - INFO - joeynmt.model - Enc-dec model built.
2021-12-03 16:52:57,073 - INFO - joeynmt.prediction - Decoding on dev set (data/enyo/dev.bpe.yo)...
2021-12-03 16:53:29,882 - INFO - joeynmt.prediction -  dev bleu[none]:   1.33 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-12-03 16:53:29,882 - INFO - joeynm